In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import pandas_profiling
import sys,os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas.api.types as ptypes
from numpy.random import choice
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import random
from random import randrange
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import  matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

In [44]:
airbnb=pd.read_csv('./amit/Airbnb/clean_train.csv')
print(df.columns)
airbnb.head()
airbnb.shape

Index(['Bathrooms', 'Bedrooms', 'Beds', 'LocationName', 'NumGuests',
       'NumReviews', 'Price', 'Rating', 'latitude', 'longitude', 'zipcode',
       'pop2016', 'pop2010', 'pop2000', 'cost_living_index (US avg. = 100)',
       'land_area (sq.mi.)', 'water_area (sq.mi.)',
       'pop_density (people per mile)', 'number of males', 'number of females',
       'prop taxes paid 2016', 'median taxes (with mortgage',
       'median taxes (no mortgage)', 'median house value',
       'median houshold income', 'median monthly owner costs (with mortgage)',
       'median monthly owner costs (no mortgage)', 'median gross rent',
       'median asking price for vacant for-sale home/condo',
       'unemployment (%)', 'Number of Homes', 'Count of Abnb',
       'Density of Abnb (%)', 'Average Abnb Price (by zipcode)',
       'Average NumReviews (by zipcode)', 'Average Rating (by zipcode)',
       'Average Number of Bathrooms (by zipcode)',
       'Average Number of Bedrooms (by zipcode)',
       'Ave

(7965, 40)

In [45]:
def load_data():
    airbnb=pd.read_csv('./amit/Airbnb/clean_train.csv')
    airbnb['Rating'] =airbnb['Rating'].apply(lambda x: 1 if x =="Y" else 0)
    y = airbnb['Price']
    X = airbnb.drop(['Price'],axis=1)
    return X,y
X,y =load_data()
feature_cols = X.columns.to_list()


In [46]:
X,y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [14]:
##forward feature selection 
sfs_top10=SFS(RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1),
       k_features=10, 
        forward=True, 
        floating=False, 
        verbose=2,
        scoring='accuracy',
        cv=4, 
       n_jobs=-1
       ).fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:  3.2min finished

[2020-07-08 22:28:00] Features: 1/10 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  38 | elapsed:    3.8s finished

[2020-07-08 22:28:04] Features: 2/10 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 out of  37 | elapsed:    3.8s finished

[2020-07-08 22:28:08] Features: 3/10 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.7s finished

[2020-07-08 22:28:12] Features: 4/10 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    3.9s finished

[2020-07-08 22:28:16] Features: 5/10 -- score: nan[Parallel(

In [47]:
print('top10 feature by forward feature selection:',sfs_top10.k_feature_names_)

print('forward section top 5 accuracy score:',sfs_top10.k_score_)

top10 feature by forward feature selection: ('Bathrooms', 'Bedrooms', 'Beds', 'LocationName', 'NumGuests', 'NumReviews', 'Rating', 'latitude', 'longitude', 'zipcode')
forward section top 5 accuracy score: nan


In [48]:
top_10 =['Bathrooms', 'Bedrooms', 'Beds', 'LocationName', 'NumGuests', 'NumReviews', 'Rating', 'latitude', 'longitude', 'zipcode']

In [49]:
airbnb.head(5)

,Bathrooms,Bedrooms,Beds,LocationName,NumGuests,NumReviews,Price,Rating,latitude,longitude,...,Number of Homes,Count of Abnb,Density of Abnb (%),Average Abnb Price (by zipcode),Average NumReviews (by zipcode),Average Rating (by zipcode),Average Number of Bathrooms (by zipcode),Average Number of Bedrooms (by zipcode),Average Number of Beds (by zipcode),Average Number of Guests (by zipcode)
0,1.0,1.0,2.0,Seattle,4.0,99.0,124.0,Y,47.61531,-122.33761,...,4886.4,143,2.926490,114.888112,74.702290,4.796154,1.076923,1.240385,1.790210,1.790210
1,2.0,4.0,4.0,Washington,8.0,186.0,330.0,Y,38.88536,-77.00601,...,11794.8,410,3.476108,127.748769,54.344444,4.900289,1.196078,1.442708,2.487805,2.487805
2,3.0,3.0,4.0,Atlanta,8.0,42.0,160.0,Y,33.74185,-84.37858,...,9114.4,200,2.194330,119.368687,40.400000,4.937500,1.375000,1.602094,1.925000,1.925000
3,1.0,1.0,2.0,San Diego,4.0,11.0,85.0,Y,32.74380,-117.16348,...,13743.2,503,3.659992,155.152305,49.580977,4.882271,1.357853,1.665944,2.151093,2.151093
4,1.0,2.0,2.0,San Francisco,4.0,12.0,255.0,N,37.76075,-122.43779,...,13593.6,423,3.111758,194.211679,85.532468,4.914729,1.201946,1.421053,1.532847,1.532847


In [50]:

airbnb['Rating'] =airbnb['Rating'].apply(lambda x: 1 if x =="Y" else 0)
#examing the changes
airbnb.head(5)

,Bathrooms,Bedrooms,Beds,LocationName,NumGuests,NumReviews,Price,Rating,latitude,longitude,...,Number of Homes,Count of Abnb,Density of Abnb (%),Average Abnb Price (by zipcode),Average NumReviews (by zipcode),Average Rating (by zipcode),Average Number of Bathrooms (by zipcode),Average Number of Bedrooms (by zipcode),Average Number of Beds (by zipcode),Average Number of Guests (by zipcode)
0,1.0,1.0,2.0,Seattle,4.0,99.0,124.0,1,47.61531,-122.33761,...,4886.4,143,2.926490,114.888112,74.702290,4.796154,1.076923,1.240385,1.790210,1.790210
1,2.0,4.0,4.0,Washington,8.0,186.0,330.0,1,38.88536,-77.00601,...,11794.8,410,3.476108,127.748769,54.344444,4.900289,1.196078,1.442708,2.487805,2.487805
2,3.0,3.0,4.0,Atlanta,8.0,42.0,160.0,1,33.74185,-84.37858,...,9114.4,200,2.194330,119.368687,40.400000,4.937500,1.375000,1.602094,1.925000,1.925000
3,1.0,1.0,2.0,San Diego,4.0,11.0,85.0,1,32.74380,-117.16348,...,13743.2,503,3.659992,155.152305,49.580977,4.882271,1.357853,1.665944,2.151093,2.151093
4,1.0,2.0,2.0,San Francisco,4.0,12.0,255.0,0,37.76075,-122.43779,...,13593.6,423,3.111758,194.211679,85.532468,4.914729,1.201946,1.421053,1.532847,1.532847


In [ ]:
airbnb.drop(['LocationName'], axis=1, inplace=True)

In [54]:
def Encode(airbnb):
    for column in airbnb.columns[airbnb.columns.isin(['LocationName'])]:
        airbnb[column] = airbnb[column].factorize()[0]
    return airbnb

newAirbnb = Encode(airbnb.copy())

In [56]:
newAirbnb.head(15)

,Bathrooms,Bedrooms,Beds,LocationName,NumGuests,NumReviews,Price,Rating,latitude,longitude,...,Number of Homes,Count of Abnb,Density of Abnb (%),Average Abnb Price (by zipcode),Average NumReviews (by zipcode),Average Rating (by zipcode),Average Number of Bathrooms (by zipcode),Average Number of Bedrooms (by zipcode),Average Number of Beds (by zipcode),Average Number of Guests (by zipcode)
0,1.0,1.0,2.0,0,4.0,99.0,124.0,1,47.61531,-122.33761,...,4886.4,143,2.926490,114.888112,74.702290,4.796154,1.076923,1.240385,1.790210,1.790210
1,2.0,4.0,4.0,1,8.0,186.0,330.0,1,38.88536,-77.00601,...,11794.8,410,3.476108,127.748769,54.344444,4.900289,1.196078,1.442708,2.487805,2.487805
2,3.0,3.0,4.0,2,8.0,42.0,160.0,1,33.74185,-84.37858,...,9114.4,200,2.194330,119.368687,40.400000,4.937500,1.375000,1.602094,1.925000,1.925000
3,1.0,1.0,2.0,3,4.0,11.0,85.0,1,32.74380,-117.16348,...,13743.2,503,3.659992,155.152305,49.580977,4.882271,1.357853,1.665944,2.151093,2.151093
4,1.0,2.0,2.0,4,4.0,12.0,255.0,0,37.76075,-122.43779,...,13593.6,423,3.111758,194.211679,85.532468,4.914729,1.201946,1.421053,1.532847,1.532847
5,1.0,2.0,2.0,5,4.0,8.0,70.0,1,29.97451,-90.08614,...,14967.6,829,5.538630,109.845499,49.687906,4.878179,1.249698,1.848285,2.343788,2.343788
6,1.0,1.0,1.0,6,2.0,50.0,100.0,0,40.76992,-73.95604,...,18364.0,47,0.255936,150.510638,44.071429,4.684211,1.042553,1.277778,1.574468,1.574468
7,2.0,1.0,1.0,7,2.0,6.0,80.0,1,40.66744,-73.98798,...,28387.2,175,0.616475,128.310345,51.337838,4.854610,1.040230,1.422360,1.914286,1.914286
8,2.0,3.0,3.0,4,5.0,15.0,599.0,1,37.75007,-122.43925,...,11502.4,365,3.173251,210.890411,72.403509,4.920354,1.257618,1.629518,1.917808,1.917808
9,2.0,1.0,4.0,1,2.0,19.0,33.0,1,38.91848,-77.02970,...,20898.4,189,0.904375,112.835979,70.791908,4.885965,1.243386,1.348993,1.793651,1.793651


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score

In [71]:
x = newAirbnb.iloc[:,[0,1,2,3,4,5]]
y = newAirbnb['Price']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1)
x_train.head()
y_train.head()

2857    215.0
825      46.0
2837    116.0
6502     62.0
781      60.0
Name: Price, dtype: float64

In [74]:
X

,Bathrooms,Bedrooms,Beds,LocationName,NumGuests,NumReviews,Rating,latitude,longitude,zipcode,...,Number of Homes,Count of Abnb,Density of Abnb (%),Average Abnb Price (by zipcode),Average NumReviews (by zipcode),Average Rating (by zipcode),Average Number of Bathrooms (by zipcode),Average Number of Bedrooms (by zipcode),Average Number of Beds (by zipcode),Average Number of Guests (by zipcode)
0,1.0,1.0,2.0,Seattle,4.0,99.0,1,47.61531,-122.33761,98101,...,4886.4,143,2.926490,114.888112,74.702290,4.796154,1.076923,1.240385,1.790210,1.790210
1,2.0,4.0,4.0,Washington,8.0,186.0,1,38.88536,-77.00601,20003,...,11794.8,410,3.476108,127.748769,54.344444,4.900289,1.196078,1.442708,2.487805,2.487805
2,3.0,3.0,4.0,Atlanta,8.0,42.0,1,33.74185,-84.37858,30312,...,9114.4,200,2.194330,119.368687,40.400000,4.937500,1.375000,1.602094,1.925000,1.925000
3,1.0,1.0,2.0,San Diego,4.0,11.0,1,32.74380,-117.16348,92103,...,13743.2,503,3.659992,155.152305,49.580977,4.882271,1.357853,1.665944,2.151093,2.151093
4,1.0,2.0,2.0,San Francisco,4.0,12.0,0,37.76075,-122.43779,94114,...,13593.6,423,3.111758,194.211679,85.532468,4.914729,1.201946,1.421053,1.532847,1.532847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7960,1.0,1.0,2.0,New York,4.0,17.0,1,40.75949,-73.96588,10022,...,12278.4,73,0.594540,154.534247,46.238095,4.708333,1.123288,1.313725,1.780822,1.780822
7961,1.0,1.0,1.0,Washington,2.0,16.0,1,38.90508,-76.98835,20002,...,24205.6,168,0.694054,95.784431,63.905063,4.881410,1.113095,1.294118,1.803571,1.803571
7962,1.0,2.0,3.0,Seattle,5.0,12.0,0,47.57734,-122.30213,98144,...,12208.4,155,1.269618,87.164474,59.880000,4.895973,1.225806,1.404110,1.729032,1.729032
7963,2.0,2.0,3.0,San Francisco,4.0,86.0,1,37.79712,-122.41548,94133,...,11154.0,143,1.282051,269.923664,62.594828,4.865079,1.135338,1.566929,2.142857,2.142857


In [75]:
y

0       124.0
1       330.0
2       160.0
3        85.0
4       255.0
        ...  
7960    180.0
7961     65.0
7962    100.0
7963    437.0
7964     65.0
Name: Price, Length: 7965, dtype: float64

In [72]:
reg=LinearRegression()
reg.fit(x_train,y_train)
y_pred=reg.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.4545436041964077